In [1]:
!pip install imageai

In [2]:
!pip install keras_retinanet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd  /content/drive/MyDrive/Capstone2/Localization/ImageAI
!ls


/content/drive/MyDrive/Capstone2/Localization/ImageAI
'Bản sao của preprocess_image.ipynb'   OpenAi.ipynb
 class-descriptions-boxable.csv        preprocess_image.ipynb
 image				       train-annotations-bbox.csv
 model				       train-images-boxable-with-rotation.csv


In [35]:
from imageai.Detection import ObjectDetection
import keras_retinanet.models
from keras_retinanet.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from keras.utils.data_utils import GeneratorEnqueuer
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import math, os
%matplotlib inline

In [47]:
model_old_weight_path = "./model/resnet50_coco_best_v2.0.1.h5"
model_update_weight_path = "./model/resnet50_coco_best_updated_v2.0.1.h5"
model_update_inference_weight_path = "./model/resnet50_coco_best_updated_inference_v2.0.1.h5"
image_path = "./image/"

In [7]:
batch_size = 32
img_generator = ImageDataGenerator().flow_from_directory(image_path, shuffle=False, batch_size = batch_size)
n_rounds = math.ceil(img_generator.samples / img_generator.batch_size)
filenames = img_generator.filenames

img_generator = GeneratorEnqueuer(img_generator)
img_generator.start()
img_generator = img_generator.get()

Found 362 images belonging to 2 classes.


In [8]:
vocab = {"/m/01g317": "Person","/m/029bxz": "Oven"}


In [9]:
rev = {}
for k,v in vocab.items():
    rev[v.lower()] = k 

In [10]:
print(len(vocab.items()))

2


In [41]:
!ls 

'Bản sao của preprocess_image.ipynb'   OpenAi.ipynb
 class-descriptions-boxable.csv        preprocess_image.ipynb
 image				       train-annotations-bbox.csv
 model				       train-images-boxable-with-rotation.csv


2021-04-26 20:49:49.741752: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-26 20:49:51.722530: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-26 20:49:51.723549: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-26 20:49:51.734836: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-26 20:49:51.734890: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fccc7d9405d0): /proc/driver/nvidia/version does not exist
2021-04-26 20:49:52.034108: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
Traceback (most recent call last):
  File "/usr/local/bin/retinanet-convert-

In [45]:
model = keras_retinanet.models.backbone('resnet50').retinanet(num_classes=80)
model.load_weights(model_old_weight_path, by_name=True)
model.save(model_update_weight_path)

In [46]:
!retinanet-convert-model ./model/resnet50_coco_best_updated_v2.0.1.h5 ./model/resnet50_coco_best_updated_inference_v2.0.1.h5

2021-04-26 20:51:11.221579: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-26 20:51:13.141487: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-26 20:51:13.142956: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-26 20:51:13.154674: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-26 20:51:13.154732: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fccc7d9405d0): /proc/driver/nvidia/version does not exist
2021-04-26 20:51:13.496091: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [48]:
execution_path = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath(model_update_inference_weight_path)
detector.loadModel()

In [49]:
custom_objects = detector.CustomObjects(person=True,oven=True)

In [50]:
print(n_rounds)

12


In [52]:
for i in range(n_rounds):
    batch = next(img_generator)
    for j, prediction in enumerate(batch):
        image = filenames[i * batch_size + j]
        detections = detector.detectObjectsFromImage(custom_objects=custom_objects,input_image=image_path+image, output_image_path="image_with_box.png", minimum_percentage_probability = 75)        
        pred_str = ""
        labels = ""
        print(detections)
        for eachObject in detections:    
            if eachObject["name"] in rev:
                pred_str += rev[eachObject["name"]] + " " + str(float(eachObject["percentage_probability"])/100)
                pred_str += " "
                labels += eachObject['name'] + ", " + str(round(float(eachObject['percentage_probability'])/100, 1)) 
                labels += " | "
        if labels != "":
            plt.figure(figsize=(12,12))
            plt.imshow(plt.imread("image_with_box.png"))
            plt.show()

            print ("Labels Detected: ")
            print (labels)
            print ()
            print ("Prediction String: ")
            print (pred_str)

    if i == 10:
        break

Output hidden; open in https://colab.research.google.com to view.